In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 23 14:40:53 2024

@author: ethankim
"""
import math
import random
import time
import numpy


AllowableWhiteSquare=[0,7,8,9,10,11,12]
AllowableBlackSquare=[0,1,2,3,4,5,6]
Decoder = [" ", chr(9817), chr(9814), chr(9816), chr(9815), chr(9813), chr(9812),chr(9823),chr(9820),chr(9822), chr(9821),chr(9819),chr(9818)]

#display_position is meant to display a board state and state who is to move.
#It is primarily meant to be used for debugging and demonstrations.
#It does not distinguish between "legal" positions and "illegal" positions
#Either if someone inputs the wrong code for a piece or the position is not
#possible in a real game.


def display_position(pos):
    x=math.sqrt(len(pos)-1)

    if x!=x//1 or x==0:

        print("Invalid Position")

    else:
       i=0
       while i<x:
           s=""
           j=0
           while j<x:
               s = s + str(Decoder[pos[int(j+i*x)]]) + " "
               j = j + 1
           print(s)
           i = i + 1
       if(pos[int(x**2)]==1):
           print("White to Move")
       elif(pos[int(x**2)]==2):
           print("Black to Move")

#All legal move functions requires us to know where the piece is.


def find_piece(pos,piece):
    y=[]
    index = 0
    for x in pos:
        if(x==piece):
            if(index!=len(pos)-1):
                y.append(index)
        index = index + 1
    return y


#Returns a list of all moves the rooke can make, ignoring check. a represents
#where the rooke is. It is made like this since now we can call upon this for
#the queen as well

def legal_rooke(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    if(len(a)!=0):
        for i in a:

            c = pos[i]

            x = i

            #The theory behind this is that the while loop will stop iterating
            #if the piece they are about to hit is white. If white then all
            #takable pieces are greater than 6. Otherwise, all takable pieces
            #are less than 6.

            while x - row_length > -1 and pos[x-row_length] in Squares:
                x = x - row_length
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())

            #We also want to stop if the piece runs into a black piece

                if(pos[x]!=0):
                    break

            x = i

            while x + row_length < len(pos)-1 and (pos[x+row_length] in Squares):
                x = x + row_length
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break

            x = i

            #Now going left. Mod is used out of convienence.

            while (x - 1) % row_length !=3 and (pos[x-1] in Squares):
                x = x - 1
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break

            x = i

            #Now going right

            while (x + 1) % row_length != 0 and (pos[x+1] in Squares):
                x = x + 1
                temp = pos.copy()
                temp[x] = c
                temp[i] = 0
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
                if(pos[x]!=0):
                    break


            #Now everything has been moved and castling is a king move so we
            #can return pos_pos

    return pos_pos


def legal_bishop(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        c = pos[i]

        x = i

        while(x - row_length - 1 > -1 and (x - row_length - 1) % row_length != 3) and (pos[x - row_length - 1] in Squares):
            x = x - row_length - 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

        x = i

        while(x - row_length + 1 > -1 and (x - row_length + 1) % row_length != 0) and (pos[x - row_length + 1] in Squares):
            x = x - row_length + 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

        x = i
        while(x + row_length + 1 < len(pos)-1 and (x + row_length + 1) % row_length != 0) and (pos[x + row_length + 1] in Squares):
            x = x + row_length + 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break


        x = i

        while(x + row_length - 1 < len(pos)-1 and (x + row_length - 1) % row_length != 3) and (pos[x + row_length - 1] in Squares):
            x = x + row_length - 1
            temp = pos.copy()
            temp[x] = c
            temp[i] = 0
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
            if(pos[x]!=0):
                break

    return pos_pos

#Now we can just declate the queen is both of the previous two functions
#combined which will be done in the legal_moves function for consistency

#The Knight shall be defined more step by step then previous ones. Because of
#the weird way it moves, we check if we're out of bounds one step at a time
#rather than all at once.

def legal_knight(pos,a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        Squares = AllowableWhiteSquare
        b = -1
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        if(i - row_length >= 0):
           if((i-row_length-1) % row_length != 3 and (i-row_length-1) % row_length != 0 and (pos[i - row_length - 2] in Squares)):
               temp = pos.copy()
               temp[i] = 0
               temp[i-row_length-2] = pos[i]
               temp[len(pos)-1] = temp[len(pos)-1] - b
               pos_pos.append(temp.copy())
           if((i-row_length+1) % row_length != 0 and (i-row_length+1) % row_length != 3 and (pos[i - row_length + 2] in Squares)):
               temp = pos.copy()
               temp[i] = 0
               temp[i-row_length+2] = pos[i]
               temp[len(pos)-1] = temp[len(pos)-1] - b
               pos_pos.append(temp.copy())
           if(i - 2 * row_length >=0):
               if((i-2*row_length-1) % row_length != 3 and (pos[i - 2*row_length - 1] in Squares)):
                   temp = pos.copy()
                   temp[i] = 0
                   temp[i-2*row_length-1] = pos[i]
                   temp[len(pos)-1] = temp[len(pos)-1] - b
                   pos_pos.append(temp.copy())
               if((i-2*row_length+1) % row_length != 0 and (pos[i - 2*row_length + 1] in Squares)):
                   temp = pos.copy()
                   temp[i] = 0
                   temp[i-2*row_length+1] = pos[i]
                   temp[len(pos)-1] = temp[len(pos)-1] - b
                   pos_pos.append(temp.copy())
        if(i + row_length < len(pos)-1):
            if((i+row_length-1) % row_length != 3 and (i+row_length-1) % row_length != 0 and (pos[i + row_length - 2] in Squares)):
                temp = pos.copy()
                temp[i] = 0
                temp[i+row_length-2] = pos[i]
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
            if((i+row_length+1) % row_length != 0 and (i+row_length+1) % row_length != 3 and (pos[i + row_length + 2] in Squares)):
                temp = pos.copy()
                temp[i] = 0
                temp[i+row_length+2] = pos[i]
                temp[len(pos)-1] = temp[len(pos)-1] - b
                pos_pos.append(temp.copy())
            if(i + 2 * row_length <len(pos)-1):
                if((i+2*row_length-1) % row_length != 3 and (pos[i + 2*row_length - 1] in Squares)):
                    temp = pos.copy()
                    temp[i] = 0
                    temp[i+2*row_length-1] = pos[i]
                    temp[len(pos)-1] = temp[len(pos)-1] - b
                    pos_pos.append(temp.copy())
                if((i+2*row_length+1) % row_length != 0 and (pos[i + 2*row_length + 1] in Squares)):
                    temp = pos.copy()
                    temp[i] = 0
                    temp[i+2*row_length+1] = pos[i]
                    temp[len(pos)-1] = temp[len(pos)-1] - b
                    pos_pos.append(temp.copy())
    return pos_pos

#Now pawns

def legal_pawn(pos,a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableBlackSquare
    else:
        b = 1
        Squares = AllowableWhiteSquare
    for i in a:
        x = i
        if(x + b * row_length > -1 and x + b * row_length < len(pos) - 1 and pos[x + b * row_length]==0):
            x = x+b*row_length
            temp = pos.copy()
            temp[i] = 0
            temp[x] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[x] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[x] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
        if((i + b * row_length - 1) % row_length != 3 and (pos[i + b * row_length-1] not in Squares)):
            x = i + b * row_length - 1
            temp = pos.copy()
            temp[i] = 0
            temp[i+b * row_length-1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length - 1] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length - 1] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
        if((i + b * row_length + 1) % row_length != 0 and (pos[i + b * row_length+1] not in Squares)):
            x = i + b * row_length + 1
            temp = pos.copy()
            temp[i] = 0
            temp[i+b * row_length+1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            if(b == -1):
                z=[]
                y = 0
                while(y<row_length):
                    z.append(0+y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length+1] = 1 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
            else:
                z=[]
                y = 0
                while(y<row_length):
                    z.append(len(pos)-2-y)
                    y = y + 1
                if(x in z):
                    j = 1
                    while (j < 5):
                        temp[i+b * row_length+1] = 7 + j
                        pos_pos.append(temp.copy())
                        j = j + 1
                else:
                    pos_pos.append(temp.copy())
    return pos_pos

#Finally, the king

def legal_king(pos, a):
    row_length=math.sqrt(len(pos)-1)
    row_length=int(row_length)
    pos_pos=[]
    if(pos[len(pos)-1]==1):
        b = -1
        Squares = AllowableWhiteSquare
    else:
        b = 1
        Squares = AllowableBlackSquare
    for i in a:
        if((i+1) % row_length != 0 and (pos[i+1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i-1) % row_length != 3 and (pos[i-1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i-row_length) > -1 and (pos[i-row_length] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i - row_length - 1) > -1 and (i - row_length - 1) % row_length != 3 and (pos[i-row_length - 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length - 1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i - row_length + 1) > -1 and (i - row_length + 1) % row_length != 0 and (pos[i-row_length + 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i-row_length +1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i+row_length) < len(pos) - 1 and (pos[i+row_length]) in Squares):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i + row_length - 1) < len(pos) - 1 and (i + row_length - 1) % row_length != 3 and (pos[i+row_length - 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length - 1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
        if((i + row_length + 1) < len(pos) - 1 and (i + row_length + 1) % row_length != 0 and (pos[i+row_length + 1] in Squares)):
            temp = pos.copy()
            temp[i] = 0
            temp[i+row_length +1] = pos[i]
            temp[len(pos)-1] = temp[len(pos)-1] - b
            pos_pos.append(temp.copy())
    return pos_pos

#Now we create a function to discover all of these moves

def legal_moves(pos):
    pos_pos=[]
    b = pos[len(pos)-1] - 1
    a = find_piece(pos, 1 + b * 6)
    pos_pos.append(legal_pawn(pos, a).copy())
    a = find_piece(pos, 2 + b * 6)
    pos_pos.append(legal_rooke(pos, a).copy())
    a = find_piece(pos, 3 + b * 6)
    pos_pos.append(legal_knight(pos, a).copy())
    a = find_piece(pos, 4 + b * 6)
    pos_pos.append(legal_bishop(pos, a).copy())
    a = find_piece(pos, 5 + b * 6)
    #We summon all queen moves here for consistency
    pos_pos.append(legal_rooke(pos, a).copy())
    pos_pos.append(legal_bishop(pos, a).copy())
    a = find_piece(pos, 6 + b * 6)
    pos_pos.append(legal_king(pos, a).copy())
    return pos_pos


#I was trying to avoid this, but I suppose I can fix it by adjusting legal
#moves later. For now, it determines if a player has moved into check (or
#failed to counter check) by determing the legal moves of the other player
#and seeing if any of them involve the king disappearing. It's ine-eficient,
#but I'll see what I can do about that later

def is_check(pos,player):
    pos_copy=pos.copy()
    if(player == 1):
        if(len(find_piece(pos, 6))==0):
            return True
        pos_copy[len(pos_copy)-1] = 2
        toBeTested = legal_moves(pos_copy)
        for a in toBeTested:
            for moves in a:
                if(len(find_piece(moves, 6))==0):
                    return True
        return False
    else:
        if(len(find_piece(pos, 12))==0):
            return True
        pos_copy[len(pos_copy)-1] = 1
        toBeTested = legal_moves(pos_copy)
        for a in toBeTested:
            for moves in a:
                if(len(find_piece(moves, 12))==0):
                    return True
        return False


#Quick function to remove checks

def remove_check(legal_moves):
    #Michael Strauss helped me with this for loop
    for piece in legal_moves:
        piece_copy = piece[:]
        for move in piece_copy:
            if(move[len(move)-1]==1):
                b = 1
                c = 2
            else:
                b = 2
                c = 1
            if(is_check(move, c)):
                piece.remove(move)




#Now the big boy. For now, a basic code will be presented to run one game.
#A more detailed thing is to come, but I am tired


'''

Initial_Position = [ 8, 9, 10, 12,7,7,7,7,1,1,1,1,2,3,4,6,1]

pos = Initial_Position

amount = 1

count = 0

Stalemate = False

while(amount > 0):
    display_position(pos)
    a = legal_moves(pos[:]).copy()

    remove_check(a)

    amount = 0

    for i in a:
            amount = len(i) + amount

    if(amount != 0):
        index = random.randint(1, amount)

        i = 0

        j = 0

        while(len(a[j]) + i < index):
            i = len(a[j]) + i
            j = j+1

        index = index - i

        prev_pos = pos.copy()


        pos = a[j][index-1]

        if(numpy.sum(prev_pos)==numpy.sum(pos)+1 or numpy.sum(prev_pos)==numpy.sum(pos)-1 or numpy.sum(prev_pos)==numpy.sum(pos)):
            count = count + 1
        else:
            count = 0

    if(count > 49):
        print("Stalemate")
        Stalemate = True
        print(count)
        break




if(not Stalemate):
    if(is_check(pos, 1)):
       print("Black Wins")
    elif (is_check(pos,2)):
       print("White Wins")
    else:
       print("Stalemate")


'''













'\n\nInitial_Position = [ 8, 9, 10, 12,7,7,7,7,1,1,1,1,2,3,4,6,1]\n\npos = Initial_Position\n\namount = 1\n\ncount = 0\n\nStalemate = False\n\nwhile(amount > 0):\n    display_position(pos)\n    a = legal_moves(pos[:]).copy()\n\n    remove_check(a)\n\n    amount = 0\n\n    for i in a:\n            amount = len(i) + amount\n\n    if(amount != 0):\n        index = random.randint(1, amount)\n\n        i = 0\n\n        j = 0\n\n        while(len(a[j]) + i < index):\n            i = len(a[j]) + i\n            j = j+1\n\n        index = index - i\n\n        prev_pos = pos.copy()\n\n\n        pos = a[j][index-1]\n\n        if(numpy.sum(prev_pos)==numpy.sum(pos)+1 or numpy.sum(prev_pos)==numpy.sum(pos)-1 or numpy.sum(prev_pos)==numpy.sum(pos)):\n            count = count + 1\n        else:\n            count = 0\n\n    if(count > 49):\n        print("Stalemate")\n        Stalemate = True\n        print(count)\n        break\n\n\n\n\nif(not Stalemate):\n    if(is_check(pos, 1)):\n       print("B

In [ ]:
'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Oct 10 15:58:03 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
import os

position_list=[]
pos=[0,0,0,0,7,7,7,7,1,1,1,1,0,0,0,0,1]
white = [6,5,4,3,2]
black = [12,11,10,9,8]



for x in black:
    pos[1] = 0
    pos[2] = 0
    pos[3] = 0
    pos[0] = x
    for y in black:
        pos[1] = 0
        pos[2] = 0
        pos[3] = 0
        if((y == 12 and pos.count(12) < 1) or (y == 11 and pos.count(11)<1) or (y != 12 and y != 11 and pos.count(y)<2)):
            pos[1] = y
            for z in black:
                pos[2] = 0
                pos[3] = 0
                if(((z == 11 and pos.count(11)<1) or (z != 12 and z != 11 and pos.count(z)<2)) and pos.count(12)==1):
                    pos[2] = z
                    for a in black:
                        pos[3] = 0
                        if(((a == 11 and pos.count(11)<1) or (a != 12 and a != 11 and pos.count(a)<2)) and pos.count(12)==1):
                            pos[3] = a
                            for i in white:
                                pos[13] = 0
                                pos[14] = 0
                                pos[15] = 0
                                pos[12] = i
                                for j in white:
                                    pos[13] = 0
                                    pos[14] = 0
                                    pos[15] = 0
                                    if((j == 6 and pos.count(6) < 1) or (j == 5 and pos.count(5)<1) or (j != 6 and j != 5 and pos.count(j)<2)):
                                        pos[13] = j
                                        for k in white:
                                            pos[14] = 0
                                            pos[15] = 0
                                            if(((k == 6 and pos.count(6) < 1) or (k == 5 and pos.count(5)<1) or (k != 6 and k != 5 and pos.count(k)<2))):
                                                pos[14] = k
                                                for l in white:
                                                    pos[15] = 0
                                                    if(((l == 6 and pos.count(6) < 1) or (l == 5 and pos.count(5)<1) or (l != 6 and l != 5 and pos.count(l)<2)) and (pos.count(6)==1 or l == 6)):
                                                        pos[15] = l
                                                        position_list.append(pos.copy())

openings = open("Openings.txt", 'w')

for pos_temp in position_list:
    print(pos_temp, file = openings)

openings.close()
'''


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 14 11:40:32 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
#from chess import is_check
#from chess import display_position
import os

def straightener(vector_of_vectors):
    vector=[]
    for bob in vector_of_vectors:
        for vec in bob:
            vector.append(vec)
    return vector


def string_to_int_int_array(string):
    string = string.strip()

    string = string.split(']')

    vector = []

    string[0]=string[0][1:]

    string = string[:len(string)-2]

    i = 0

    for s in string:
        string_copy = string[i]
        string_copy = string_copy.split(',')
        if(i == 0):
            string_copy[0] = int(string_copy[0][1:])
        else:
            string_copy = string_copy[1:]
            string_copy[0] = int(string_copy[0][2:])
        j = 0
        for object in string_copy:
            if(j>0):
                string_copy[j] = int(string_copy[j])
            j = j + 1
        vector.append(string_copy)
        i = i + 1
    return vector

#Plan: For each pos in positions this cuntion creates a file listing the possible
#moves. If that file already exits, then it will instead read that file. This
#function will then go recursively. If all solutions are found to be bad from
#a certainn point it will return that result. Otherwise, it will burn access to
#those branches and return a benificial or alright result. We'll mark these results
#in a vector in the return statement. Hopefully will work? I wonder what this
#is called.

def solver(positions, maximum = 0):
    print_copy = positions[:]
    player = positions[0][16]
    index = 0
    vector = []
    Stalemate_count = 0
    for pos in positions:
        if(os.path.isfile("/content/moves/" + str(pos)+".txt")):
            done = open("/content/moves/" + str(pos)+".txt",'r')
            for line in done:
                if(line == "Stalemate\n"):
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
                elif(line == "Black Wins\n"):
                    return [0,0,1,False]
                elif(line == "White Wins\n"):
                    return [1,0,0,False]
                else:
                    line = string_to_int_int_array(line)
                    vector.append(line)
            done.close()
        else:
            record = open("/content/moves/" + str(pos)+".txt",'w')
            temp = legal_moves(pos)
            remove_check(temp)
            amount = 0
            for j in temp:
                    amount = len(j) + amount
            if(amount==0):
                if(is_check(pos, 1)):
                   print("Black Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([0,0,1,False])
                elif (is_check(pos,2)):
                   print("White Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([1,0,0,False])
                else:
                    print("Stalemate", file = record)
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
            else:
                temp = straightener(temp)
                print(temp, file = record)
                vector.append(temp)
            record.close()
            index = index + 1
    if(maximum<=0):
        return([0,0,0,True])
    Loss_count = 0
    k = 0
    for pos in vector:
        a = solver(pos,maximum - 1)
        record = open("/content/moves/" + str(positions[k])+".txt", 'w')
        print(pos, file = record)
        record.close()
        if(player == 1 and a[2] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([0,0,1,False])
        elif(player == 2 and a[0] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([1,0,0,False])
        elif(a[0] > 0 or a[2]>0):
            Loss_count = Loss_count + 1
        elif(a[1] > 0):
            Stalemate_count = Stalemate_count + 1
            Stalemate_pos = k
        k = k + 1
    if(Stalemate_count + Loss_count == len(positions)):
        if(Stalemate_count > 0):
            copy = positions[:]
            for i in range(0,Stalemate_pos):
                positions.remove(copy[i])
            for i in range(Stalemate_pos+1, len(copy)):
                positions.remove(copy[i])
            return([0,1,0,False])
        else:
            if(player == 1):
                return([1,0,0,False])
            else:
                return([0,0,1,False])
    else:
        return([0,0,0,True])






In [ ]:
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array

Openings_file = open("Openings.txt", 'r')
Openings = []
for line in Openings_file:
    Openings.append(string_to_int_array(line))

Openings_file.close()
len(Openings)

9840

In [ ]:
Openings_file = open("White_Wins_in_one.txt", 'r')
count = 0
for line in Openings_file:
    count = count + 1
Openings_file.close()
count

7687

In [ ]:
import os
import shutil

if os.path.exists("/content/moves/"):
    shutil.rmtree("/content/moves/")
os.mkdir("/content/moves/")

In [ ]:
Openings = [[8, 12,11,8,7,7,7,7,1,1,1,1,2,6,5,2,1]]
len(Openings)
print(Openings)

[[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]]


In [ ]:
'''
#from BasicSolver import solver
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array


Unsolved = []
Solved_White_Wins = []
Solved_Black_Wins = []
Solved_Stalemate = []

for opening in Openings:
    temp = [opening]
    solver(temp,1)
    solver(temp,2)
    solver(temp,3)
    solver(temp,4)
    a = solver(temp,7)
    if(a[3]):
        Unsolved.append(opening)
    elif(a[0]==1):
        Solved_White_Wins.append(opening)
    elif(a[1]==1):
        Solved_Stalemate.append(opening)
    else:
        Solved_Black_Wins.append(opening)


Openings_file = open("Openings.txt", 'w')
for opening in Unsolved:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("White_Wins_in_six.txt", 'w')
for opening in Solved_White_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("Black_Wins_in_six1.txt", 'w')
for opening in Solved_Black_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("Stalemate_in_six2.txt", 'w')
for opening in Solved_Stalemate:
    print(opening, file = Openings_file)
Openings_file.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
import os
#import shutil
#shutil.rmtree('/content/drive/My Drive/[10,12,10,11,7,7,7,7,1,1,1,1,6,4,3,5,1]')
os.mkdir('/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]')

In [ ]:
'''
import shutil
shutil.move('/content/moves', '/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]')

'/content/drive/My Drive/[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1]/moves'

To get the test cases, start with opening 1 in openings.txt exists before merge. We'll choose a variety of unsolved things and hope we don't get unlucky and find one that is solved in 5 and not too many are found. \\
First position: \\
[10, 12, 10, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 4, 3, 5, 1] \\
solved in seven  \\
Second Position: \\
[9, 12, 9, 8, 7, 7, 7, 7, 1, 1, 1, 1, 3, 6, 4, 5, 1] \\
solved in nine \\
Third Position: \\
[8, 12, 8, 9, 7, 7, 7, 7, 1, 1, 1, 1, 2, 2, 4, 6, 1] \\
solved in seven \\
Fourth Position: \\
[12, 11, 10, 10, 7, 7, 7, 7, 1, 1, 1, 1, 6, 5, 4, 4, 1] \\
solved in nine \\
Fifth Position: \\
[12, 11, 9, 8, 7, 7, 7, 7, 1, 1, 1, 1, 5, 6, 4, 3, 1] \\
solved in nine \\
Sixth Position: \\
[12, 11, 8, 9, 7, 7, 7, 7, 1, 1, 1, 1, 3, 2, 6, 5, 1] \\
solved in seven \\
Seventh Position: \\
[12, 10, 10, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 4, 2, 2, 1] \\
solved in seven \\
Eighth Position: \\
[12, 9, 10, 9, 7, 7, 7, 7, 1, 1, 1, 1, 4, 6, 5, 4, 1] \\
solved in nine \\
Ninth Position: \\
[12, 8, 8, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 3, 5, 3, 1] \\
solved in nine \\
Tenth Position: \\
[8, 12, 11, 8, 7, 7, 7, 7, 1, 1, 1, 1, 2, 6, 5, 2, 1] \\
solved in seven \\
FROM WIKIPEDIA


In [ ]:
'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 14 11:40:32 2024

@author: ethankim
"""
#from chess import legal_moves
#from chess import remove_check
#from chess import is_check
#from chess import display_position
import os



#Plan: For each pos in positions this cuntion creates a file listing the possible
#moves. If that file already exits, then it will instead read that file. This
#function will then go recursively. If all solutions are found to be bad from
#a certainn point it will return that result. Otherwise, it will burn access to
#those branches and return a benificial or alright result. We'll mark these results
#in a vector in the return statement. Hopefully will work? I wonder what this
#is called.

def memoryless_solver(positions, maximum = 0):
    print_copy = positions[:]
    player = positions[0][16]
    index = 0
    vector = []
    Stalemate_count = 0
    for pos in positions:
        if(os.path.isfile("/content/moves/" + str(pos)+".txt")):
            done = open("/content/moves/" + str(pos)+".txt",'r')
            for line in done:
                if(line == "Stalemate\n"):
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
                elif(line == "Black Wins\n"):
                    return [0,0,1,False]
                elif(line == "White Wins\n"):
                    return [1,0,0,False]
                else:
                    line = string_to_int_int_array(line)
                    vector.append(line)
            done.close()
        else:
            record = open("/content/moves/" + str(pos)+".txt",'w')
            temp = legal_moves(pos)
            remove_check(temp)
            amount = 0
            for j in temp:
                    amount = len(j) + amount
            if(amount==0):
                if(is_check(pos, 1)):
                   print("Black Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([0,0,1,False])
                elif (is_check(pos,2)):
                   print("White Wins", file = record)
                   copy = positions[:]
                   for i in range(0,index):
                       positions.remove(copy[i])
                   for i in range(index+1, len(copy)):
                       positions.remove(copy[i])
                   return([1,0,0,False])
                else:
                    print("Stalemate", file = record)
                    Stalemate_count = Stalemate_count + 1
                    Stalemate_pos = index
            else:
                temp = straightener(temp)
                print(temp, file = record)
                vector.append(temp)
            record.close()
            index = index + 1
    if(maximum<=0):
        return([0,0,0,True])
    Loss_count = 0
    k = 0
    for pos in vector:
        a = memoryless_solver(pos,maximum - 1)
        record = open("/content/moves/" + str(positions[k])+".txt", 'w')
        print(pos, file = record)
        record.close()
        if(player == 1 and a[2] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([0,0,1,False])
        elif(player == 2 and a[0] > 0):
            copy = positions[:]
            for i in range(0,k):
                positions.remove(copy[i])
            for i in range(k+1, len(copy)):
                positions.remove(copy[i])
            return([1,0,0,False])
        elif(a[0] > 0 or a[2]>0):
            Loss_count = Loss_count + 1
        elif(a[1] > 0):
            Stalemate_count = Stalemate_count + 1
            Stalemate_pos = k
        k = k + 1
    if(Stalemate_count + Loss_count == len(positions)):
        if(Stalemate_count > 0):
            copy = positions[:]
            for i in range(0,Stalemate_pos):
                positions.remove(copy[i])
            for i in range(Stalemate_pos+1, len(copy)):
                positions.remove(copy[i])
            return([0,1,0,False])
        else:
            if(player == 1):
                return([1,0,0,False])
            else:
                return([0,0,1,False])
    else:
        return([0,0,0,True])






In [ ]:
'''
#from BasicSolver import memoryless_solver
import shutil
def string_to_int_array(s):
    s=s.strip()
    s=s.split(',')
    Array=[]
    Array.append(int(s[0][1:]))
    for i in range(1,len(s)-1):
        Array.append(int(s[i]))
    Array.append(int(s[len(s)-1][1:2]))
    return Array

Unsolved = []
Solved_White_Wins = []
Solved_Black_Wins = []
Solved_Stalemate = []

for opening in Openings:
    temp = [opening]
    a = memoryless_solver(temp,4)
    if(a[3]):
        Unsolved.append(opening)
    elif(a[0]==1):
        Solved_White_Wins.append(opening)
    elif(a[1]==1):
        Solved_Stalemate.append(opening)
    else:
        Solved_Black_Wins.append(opening)
    shutil.rmtree('/content/moves/')
    os.mkdir("/content/moves/")


Openings_file = open("/content/drive/My Drive/Openings.txt", 'w')
for opening in Unsolved:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/White_Wins_in_four.txt", 'w')
for opening in Solved_White_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/Black_Wins_in_four.txt", 'w')
for opening in Solved_Black_Wins:
    print(opening, file = Openings_file)
Openings_file.close()

Openings_file = open("/content/drive/My Drive/Stalemate_in_four.txt", 'w')
for opening in Solved_Stalemate:
    print(opening, file = Openings_file)
Openings_file.close()

First round, 280 completed. Fixed problem so hopefully will manage to run the rest.

Note to self. Might have to restart it eventually since made an error in solver. Error is fine, just have to check there weren't any stalemates in the early stages at some point.

### Deep Reinforcement Learning for 4x4 Minichess Variants Based on AlphaZero ###

In this implementation, we referenced the following paper:

**Hongming Zhang and Tianyang Yu**, *AlphaZero* DOI: [https://doi.org/10.1007/978-981-15-4095-0_15](https://doi.org/10.1007/978-981-15-4095-0_15)


In [ ]:
"""
Deep Reinforcment Learning for 4x4 Mini Chess Variants based on AlphaZero

@author: Sanghun Han
"""
# Prepare program

import subprocess
import sys

try:
    import numpy as np
except ImportError:
    print("numpy not found, installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy"])
    import numpy as np

try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim
except ImportError:
    print("torch not found, installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "torch"])
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torch.optim as optim

try:
    import matplotlib.pyplot as plt
except ImportError:
    print("matplotlib not found, installing...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib"])
    import matplotlib.pyplot as plt

import math
import time
import random
import os
import ast

In [ ]:
# Configuration Settings
MCTS_SIMULATIONS = 800
NUM_RES_BLOCKS = 10
BATCH_SIZE = 384
LEARNING_RATE = 0.01
MOMENTUM = 0.05
ACTION_SIZE = 384

BOARD_SIZE = 4  # 4x4 board
TOTAL_SQUARES = BOARD_SIZE * BOARD_SIZE  # 16 squares

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# Define piece codes
# White pieces: 1-6
# Black pieces: 7-12
PIECE_CODES = {
    'empty': 0,
    'w_pawn': 1,
    'w_rook': 2,
    'w_knight': 3,
    'w_bishop': 4,
    'w_queen': 5,
    'w_king': 6,
    'b_pawn': 7,
    'b_rook': 8,
    'b_knight': 9,
    'b_bishop': 10,
    'b_queen': 11,
    'b_king': 12,
}

# Direction vectors for piece movements
DIRECTIONS = {
    'rook': [(-1, 0), (1, 0), (0, -1), (0, 1)],
    'bishop': [(-1, -1), (-1, 1), (1, -1), (1, 1)],
    'queen': [(-1, 0), (1, 0), (0, -1), (0, 1),
              (-1, -1), (-1, 1), (1, -1), (1, 1)],
    'knight': [(-2, -1), (-2, 1), (-1, -2), (-1, 2),
               (1, -2), (1, 2), (2, -1), (2, 1)],
    'king': [(-1, -1), (-1, 0), (-1, 1),
             (0, -1),          (0, 1),
             (1, -1),  (1, 0), (1, 1)],
}

# Mapping from piece code to piece type
PIECE_TYPE = {
    1: 'pawn',
    2: 'rook',
    3: 'knight',
    4: 'bishop',
    5: 'queen',
    6: 'king',
    7: 'pawn',
    8: 'rook',
    9: 'knight',
    10: 'bishop',
    11: 'queen',
    12: 'king',
}

In [ ]:
# Build action space
def build_action_space():
    action_space = {}
    index = 0
    for from_sq in range(TOTAL_SQUARES):
        for to_sq in range(TOTAL_SQUARES):
            action_space[(from_sq, to_sq, 0)] = index
            index += 1
            # Add promotion actions
            from_rank = from_sq // BOARD_SIZE
            to_rank = to_sq // BOARD_SIZE
            if from_rank == 1 and to_rank == 0:
                # White pawn promotion
                for promo_piece in [PIECE_CODES['w_queen'], PIECE_CODES['w_rook'],
                                    PIECE_CODES['w_bishop'], PIECE_CODES['w_knight']]:
                    action_space[(from_sq, to_sq, promo_piece)] = index
                    index += 1
            elif from_rank == 2 and to_rank == 3:
                # Black pawn promotion
                for promo_piece in [PIECE_CODES['b_queen'], PIECE_CODES['b_rook'],
                                    PIECE_CODES['b_bishop'], PIECE_CODES['b_knight']]:
                    action_space[(from_sq, to_sq, promo_piece)] = index
                    index += 1
    #print(f"Action space size: {len(action_space)}")
    return action_space

# Initialize action space
ACTION_SPACE = build_action_space()
ACTION_SIZE = len(ACTION_SPACE)

# Game logic functions
def get_initial_state():
    """
    Returns the initial game state.
    """
    # Index 0-15 represent the squares, index 16 is the player to move (1 for White)
    # Starting position:
    pos = [10, 12, 10, 11, 7, 7, 7, 7, 1, 1, 1, 1, 6, 4, 3, 5, 1]
    return pos

def encode_board(pos):
    """
    Encodes the board position into a 12x4x4 tensor.
    Each channel corresponds to a piece type.
    """
    board_tensor = np.zeros((12, BOARD_SIZE, BOARD_SIZE), dtype=np.float32)
    for idx in range(TOTAL_SQUARES):
        piece = pos[idx]
        if piece != 0:
            piece_idx = piece - 1
            row = idx // BOARD_SIZE
            col = idx % BOARD_SIZE
            board_tensor[piece_idx, row, col] = 1.0
    return board_tensor

def apply_action(pos, action):
    """
    Applies the action to the position and returns the new position.
    """
    from_sq, to_sq, promotion_piece = action
    new_pos = pos.copy()

    moving_piece = new_pos[from_sq]
    # Move the piece
    new_pos[to_sq] = moving_piece
    new_pos[from_sq] = 0

    # Handle promotion
    if promotion_piece != 0:
        new_pos[to_sq] = promotion_piece

    # Switch player turn
    new_pos[16] = 2 if new_pos[16] == 1 else 1

    return new_pos

def get_piece_positions(pos, player_pieces):
    """
    Returns a list of tuples (piece_code, position_index) for all pieces belonging to the player.
    """
    pieces = []
    for idx in range(TOTAL_SQUARES):
        piece = pos[idx]
        if piece in player_pieces:
            pieces.append((piece, idx))
    return pieces

def generate_sliding_moves(pos, piece_code, position_idx, directions):
    """
    Generates moves for sliding pieces (rook, bishop, queen).
    """
    moves = []
    player = 1 if piece_code <= 6 else 2
    opponent_pieces = range(7, 13) if player == 1 else range(1, 7)

    row = position_idx // BOARD_SIZE
    col = position_idx % BOARD_SIZE

    for dr, dc in directions:
        r, c = row + dr, col + dc
        while 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
            idx = r * BOARD_SIZE + c
            target_piece = pos[idx]
            if target_piece == 0:
                moves.append((position_idx, idx, 0))
            elif target_piece in opponent_pieces:
                moves.append((position_idx, idx, 0))
                break
            else:
                break
            r += dr
            c += dc

    return moves

def generate_pawn_moves(pos, piece_code, position_idx):
    """
    Generates moves for pawns, including promotions.
    """
    moves = []
    player = 1 if piece_code <= 6 else 2
    opponent_pieces = range(7, 13) if player == 1 else range(1, 7)
    direction = -1 if player == 1 else 1

    row = position_idx // BOARD_SIZE
    col = position_idx % BOARD_SIZE

    # Forward move
    r, c = row + direction, col
    if 0 <= r < BOARD_SIZE:
        idx = r * BOARD_SIZE + c
        if pos[idx] == 0:
            # Promotion
            if (player == 1 and r == 0) or (player == 2 and r == BOARD_SIZE - 1):
                promo_pieces = [PIECE_CODES['w_queen'], PIECE_CODES['w_rook'],
                                PIECE_CODES['w_bishop'], PIECE_CODES['w_knight']] if player == 1 else \
                               [PIECE_CODES['b_queen'], PIECE_CODES['b_rook'],
                                PIECE_CODES['b_bishop'], PIECE_CODES['b_knight']]
                for promo_piece in promo_pieces:
                    moves.append((position_idx, idx, promo_piece))
            else:
                moves.append((position_idx, idx, 0))

    # Captures
    for dc in [-1, 1]:
        r, c = row + direction, col + dc
        if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
            idx = r * BOARD_SIZE + c
            target_piece = pos[idx]
            if target_piece in opponent_pieces:
                # Promotion
                if (player == 1 and r == 0) or (player == 2 and r == BOARD_SIZE - 1):
                    promo_pieces = [PIECE_CODES['w_queen'], PIECE_CODES['w_rook'],
                                    PIECE_CODES['w_bishop'], PIECE_CODES['w_knight']] if player == 1 else \
                                   [PIECE_CODES['b_queen'], PIECE_CODES['b_rook'],
                                    PIECE_CODES['b_bishop'], PIECE_CODES['b_knight']]
                    for promo_piece in promo_pieces:
                        moves.append((position_idx, idx, promo_piece))
                else:
                    moves.append((position_idx, idx, 0))

    return moves

def generate_knight_moves(pos, piece_code, position_idx):
    """
    Generates moves for knights.
    """
    moves = []
    player = 1 if piece_code <= 6 else 2
    opponent_pieces = range(7, 13) if player == 1 else range(1, 7)

    row = position_idx // BOARD_SIZE
    col = position_idx % BOARD_SIZE

    for dr, dc in DIRECTIONS['knight']:
        r, c = row + dr, col + dc
        if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
            idx = r * BOARD_SIZE + c
            target_piece = pos[idx]
            if target_piece == 0 or target_piece in opponent_pieces:
                moves.append((position_idx, idx, 0))

    return moves

def generate_king_moves(pos, piece_code, position_idx):
    """
    Generates moves for the king.
    """
    moves = []
    player = 1 if piece_code <= 6 else 2
    opponent_pieces = range(7, 13) if player == 1 else range(1, 7)

    row = position_idx // BOARD_SIZE
    col = position_idx % BOARD_SIZE

    for dr, dc in DIRECTIONS['king']:
        r, c = row + dr, col + dc
        if 0 <= r < BOARD_SIZE and 0 <= c < BOARD_SIZE:
            idx = r * BOARD_SIZE + c
            target_piece = pos[idx]
            if target_piece == 0 or target_piece in opponent_pieces:
                moves.append((position_idx, idx, 0))

    return moves

def legal_moves(pos):
    """
    Generates all possible legal moves for the current player in the given position.
    Returns a dictionary mapping action tuples to new positions.
    """
    current_player = pos[16]
    player_pieces = range(1, 7) if current_player == 1 else range(7, 13)
    legal_actions = {}

    pieces = get_piece_positions(pos, player_pieces)
    all_moves = []

    for piece_code, position_idx in pieces:
        piece_type = PIECE_TYPE[piece_code]
        if piece_type == 'pawn':
            moves = generate_pawn_moves(pos, piece_code, position_idx)
        elif piece_type == 'rook':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['rook'])
        elif piece_type == 'bishop':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['bishop'])
        elif piece_type == 'queen':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['queen'])
        elif piece_type == 'knight':
            moves = generate_knight_moves(pos, piece_code, position_idx)
        elif piece_type == 'king':
            moves = generate_king_moves(pos, piece_code, position_idx)
        else:
            moves = []
        all_moves.extend(moves)

    # Remove moves that leave the player in check
    for action in all_moves:
        new_pos = apply_action(pos, action)
        if not is_check(new_pos, current_player):
            legal_actions[action] = new_pos

    return legal_actions

def is_check(pos, player):
    """
    Returns True if the player's king is in check, False otherwise.
    """
    opponent = 2 if player == 1 else 1
    opponent_pieces = range(7, 13) if opponent == 2 else range(1, 7)
    player_king_code = PIECE_CODES['w_king'] if player == 1 else PIECE_CODES['b_king']

    # Find the king's position
    try:
        king_idx = pos.index(player_king_code)
    except ValueError:
        # King is not on the board (captured)
        return True

    # Generate all possible moves for the opponent
    opponent_moves = []

    pieces = get_piece_positions(pos, opponent_pieces)
    for piece_code, position_idx in pieces:
        piece_type = PIECE_TYPE[piece_code]
        if piece_type == 'pawn':
            moves = generate_pawn_moves(pos, piece_code, position_idx)
        elif piece_type == 'rook':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['rook'])
        elif piece_type == 'bishop':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['bishop'])
        elif piece_type == 'queen':
            moves = generate_sliding_moves(pos, piece_code, position_idx, DIRECTIONS['queen'])
        elif piece_type == 'knight':
            moves = generate_knight_moves(pos, piece_code, position_idx)
        elif piece_type == 'king':
            moves = generate_king_moves(pos, piece_code, position_idx)
        else:
            moves = []
        opponent_moves.extend(moves)

    # Check if any of the opponent's moves can capture the king
    for _, to_idx, _ in opponent_moves:
        if to_idx == king_idx:
            return True

    return False

def get_game_status(pos, move_count=0, position_history=None):
    """
    Determine the status of the game based on the current board position and draw conditions.

    Parameters:
    pos (list or tuple): The current position of the game, including player turn at index 16.
    move_count (int): The number of consecutive moves without a pawn move or capture.
    position_history (dict): A dictionary tracking the number of times each position has occurred.

    Returns:
    tuple: A tuple indicating whether the game is over, the winner (if any), and the reason.
           - (True, winner, 'checkmate') if there's a checkmate
           - (True, 0, 'stalemate') if there's a stalemate
           - (True, 0, 'fifty-move rule') if the fifty-move rule is triggered
           - (True, 0, 'threefold repetition') if threefold repetition is triggered
           - (True, 0, 'insufficient material') if there’s insufficient material
           - (False, None, None) if the game is ongoing
    """
    current_player = pos[16]
    legal_actions = legal_moves(pos)

    if not legal_actions:
        if is_check(pos, current_player):
            # Checkmate
            winner = 2 if current_player == 1 else 1
            return True, winner, 'checkmate'
        else:
            # Stalemate
            return True, 0, 'stalemate'

    # Fifty-move rule
    if move_count >= 50:
        return True, 0, 'fifty-move rule'

    # Threefold repetition
    if position_history is not None:
        position_key = tuple(pos)  # Convert pos to a hashable type for tracking
        position_history[position_key] = position_history.get(position_key, 0) + 1
        if position_history[position_key] >= 3:
            return True, 0, 'threefold repetition'

    # Insufficient material (e.g., only kings left)
    if has_insufficient_material(pos):
        return True, 0, 'insufficient material'

    # Game is still ongoing
    return False, None, None

def has_insufficient_material(pos):
    """
    Check if there is insufficient material to continue the game.

    Parameters:
    pos (list or tuple): The current position of the game.

    Returns:
    bool: True if both players have insufficient material to continue the game, otherwise False.
    """
    # Initialize counts for both white and black
    white_counts = {"kings": 0, "bishops": 0, "knights": 0, "other": 0, "bishop_colors": set()}
    black_counts = {"kings": 0, "bishops": 0, "knights": 0, "other": 0, "bishop_colors": set()}

    # Loop through the board and count pieces
    for idx, square in enumerate(pos[:16]):  # Adjust range based on actual board length (4x4 board in this case)
        row, col = idx // BOARD_SIZE, idx % BOARD_SIZE

        if square == PIECE_CODES['w_king']:
            white_counts["kings"] += 1
        elif square == PIECE_CODES['w_bishop']:
            white_counts["bishops"] += 1
            white_counts["bishop_colors"].add((row + col) % 2)  # Track bishop's color
        elif square == PIECE_CODES['w_knight']:
            white_counts["knights"] += 1
        elif square in range(1, 7):  # Other white pieces
            white_counts["other"] += 1

        elif square == PIECE_CODES['b_king']:
            black_counts["kings"] += 1
        elif square == PIECE_CODES['b_bishop']:
            black_counts["bishops"] += 1
            black_counts["bishop_colors"].add((row + col) % 2)  # Track bishop's color
        elif square == PIECE_CODES['b_knight']:
            black_counts["knights"] += 1
        elif square in range(7, 13):  # Other black pieces
            black_counts["other"] += 1

    # White has only a king
    if white_counts["other"] == 0 and white_counts["bishops"] == 0 and white_counts["knights"] == 0:
        # Check if black has insufficient material
        black_insufficient = (
            black_counts["other"] == 0 and
            ((black_counts["bishops"] == 1 and black_counts["knights"] == 0 and len(black_counts["bishop_colors"]) == 1) or  # Single bishop on one color
             (black_counts["knights"] == 1 and black_counts["bishops"] == 0) or  # King and knight only
             (black_counts["bishops"] == 2 and len(black_counts["bishop_colors"]) == 1) or  # Two bishops on same color
             (black_counts["bishops"] == 0 and black_counts["knights"] == 0))  # Only king
        )
        return black_insufficient

    # Black has only a king
    if black_counts["other"] == 0 and black_counts["bishops"] == 0 and black_counts["knights"] == 0:
        # Check if white has insufficient material
        white_insufficient = (
            white_counts["other"] == 0 and
            ((white_counts["bishops"] == 1 and white_counts["knights"] == 0 and len(white_counts["bishop_colors"]) == 1) or  # Single bishop on one color
             (white_counts["knights"] == 1 and white_counts["bishops"] == 0) or  # King and knight only
             (white_counts["bishops"] == 2 and len(white_counts["bishop_colors"]) == 1) or  # Two bishops on same color
             (white_counts["bishops"] == 0 and white_counts["knights"] == 0))  # Only king
        )
        return white_insufficient

    # If neither player has just a king, return False
    return False

### Residual Neural Network ###

In [ ]:
# Residual block definition
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)  # Corrected line
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x  # Save input for skip connection
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual  # Add input (skip connection)
        return F.relu(x)

# AlphaZeroNet with 10 Residual Blocks
class AlphaZeroNet(nn.Module):
    def __init__(self, action_size=ACTION_SIZE, num_res_blocks=NUM_RES_BLOCKS):
        super(AlphaZeroNet, self).__init__()

        # Initial convolution
        self.conv1 = nn.Conv2d(12, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)

        # Residual blocks
        self.res_blocks = nn.ModuleList([ResidualBlock(64, 64) for _ in range(num_res_blocks)])

        # Fully connected layer after the residual blocks
        self.fc = nn.Linear(64 * 4 * 4, 256)
        self.bn_fc = nn.BatchNorm1d(256)

        # Policy head
        self.policy_head = nn.Linear(256, action_size)

        # Value head
        self.value_head = nn.Linear(256, 1)

    def forward(self, x):
            x = self.conv1(x)
            if x.size(0) > 1:  # Check batch size
                x = F.relu(self.bn1(x))
            else:
                x = F.relu(x)

            for res_block in self.res_blocks:
                x = res_block(x)

            x = x.view(-1, 64 * 4 * 4)
            x = self.fc(x)
            if x.size(0) > 1:
                x = F.relu(self.bn_fc(x))
            else:
                x = F.relu(x)

            # Policy head
            p = self.policy_head(x)
            p = F.softmax(p, dim=1)

            # Value head
            v = self.value_head(x)
            v = torch.tanh(v)

            return p, v

### Monte Carlo Tree Search (MCTS) ###

In [ ]:
class MCTSNode:
    def __init__(self, pos, parent=None, prior_p=1.0):
        self.pos = pos        # Game state at this node
        self.parent = parent  # Parent node
        self.children = {}    # Mapping from action to child node
        self.N = 0            # Visit count
        self.W = 0            # Total action-value
        self.Q = 0            # Mean action-value
        self.P = prior_p      # Prior probability from the policy network

    def is_leaf(self):
        return len(self.children) == 0

class MCTS:
    def __init__(self, net, cpuct=5, n_simulations=MCTS_SIMULATIONS):
        self.net = net
        self.cpuct = cpuct
        self.n_simulations = n_simulations
        self.action_space = ACTION_SPACE

    def action_to_index(self, action):
        # Converts an action tuple to its index in the action space
        return self.action_space.get(action, None)

    def index_to_action(self, index):
        # Converts an index back to its action tuple
        for action, idx in self.action_space.items():
            if idx == index:
                return action
        return None

    def search(self, root):
        # Set the network to evaluation mode
        self.net.eval()

        # Check if the root node is a terminal node
        game_over, winner, _ = get_game_status(root.pos)
        if game_over:
            # Assign value to the root node based on the game outcome
            root.Q = 1 if winner == root.pos[16] else -1 if winner else 0  # Win, loss, or draw
            root.N += 1  # Increment visit count for root
            print('Root node is a terminal state.')
            return None  # No moves to return

        # Expand the root node
        self.expand_and_evaluate(root)

        # Apply Dirichlet noise at the root for better exploration
        self.apply_dirichlet_noise(root)
        for sim in range(self.n_simulations):
            node = root
            path = [(node, None)]

            while not node.is_leaf():
                # Select the child with the highest UCB score
                max_ucb = float('-inf')
                best_action, best_child = None, None
                for action, child in node.children.items():
                    ucb = child.Q + self.cpuct * child.P * np.sqrt(node.N) / (1 + child.N)
                    if ucb > max_ucb:
                        max_ucb, best_action, best_child = ucb, action, child
                if best_child is None:
                    print('No valid child found during selection.')
                    break  # Exit the loop if no valid child is found
                node = best_child
                path.append((node, best_action))

                # Check if the node is a terminal state
                game_over, winner, _ = get_game_status(node.pos)
                if game_over:
                    value = 1 if winner == node.pos[16] else -1 if winner else 0
                    self.backup(path, value, best_action)
                    break
            else:
                # Expand and evaluate the leaf node
                value = self.expand_and_evaluate(node)
                # Backup the result up the path
                self.backup(path, value, best_action)

        # Compute the final move probabilities
        pi = np.zeros(len(self.action_space))

        for action, child in root.children.items():
            index = self.action_to_index(action)
            if index is not None:
                pi[index] = child.N
        pi = pi / pi.sum() if pi.sum() > 0 else None

        # Set the network back to training mode
        self.net.train()

        return pi

    def expand_and_evaluate(self, node):
        # Get legal actions and new positions
        legal_actions = legal_moves(node.pos)

        # Use the neural network to get policy and value
        state_tensor = torch.tensor(encode_board(node.pos)).unsqueeze(0).float()
        self.net.eval()  # Ensure the network is in eval mode
        with torch.no_grad():
            p, v = self.net(state_tensor)
        p = p.squeeze(0).numpy()
        v = v.item()

        # Mask out invalid actions
        valid_indices = [self.action_to_index(action) for action in legal_actions.keys()]
        valid_indices = [idx for idx in valid_indices if idx is not None]
        p_masked = np.zeros_like(p)
        p_masked[valid_indices] = p[valid_indices]
        total_p = np.sum(p_masked)

        # Normalize the probabilities
        if total_p > 0:
            p_masked = p_masked / total_p
        else:
            # Assign uniform probability over valid moves
            p_masked[valid_indices] = 1 / len(valid_indices)

        # Set up children nodes
        for action, new_pos in legal_actions.items():
            index = self.action_to_index(action)
            if index is not None:
                prior_p = p_masked[index]
                child_node = MCTSNode(new_pos, parent=node, prior_p=prior_p)
                node.children[action] = child_node
        return v

    def backup(self, path, value, action):
        # Update the statistics along the path
        for node, _ in reversed(path):
            node.N += 1
            node.W += -value
            node.Q = node.W / node.N
            #print(f"Backing up node: {node.pos}, Action: {action}, Value: {value}, Visits: {node.N}, Q: {node.Q}")
            value = -value  # Switch perspective

    def apply_dirichlet_noise(self, node):
        epsilon = 0.25
        alpha = 0.03  # Adjust alpha as needed
        num_actions = len(node.children)
        noise = np.random.dirichlet([alpha] * num_actions)
        for idx, (action, child) in enumerate(node.children.items()):
            child.P = (1 - epsilon) * child.P + epsilon * noise[idx]

    def print_child_node_visits(self, node):
        """
        Prints the number of visits (N) for each child node of the given parent node.

        Parameters:
        parent_node (MCTSNode): The parent node whose children visits need to be printed.
        """
        if not node.children:
            print("No child nodes found for this parent node.")
            return

        print(f"Number of visits for each child of the parent node (position: {node.pos}):")
        for action, child in node.children.items():
            from_sq, to_sq, promotion = action
            print(f"Move from {from_sq} to {to_sq} (Promotion: {promotion}) -> Visits (N): {child.N} -> Total Action Value (W): {child.W}")

### Self_play, Train_Network, and Evaluate_Network ###

In [ ]:
# Self-play and training functions
def self_play(Initial_Position, num_games, net):
    training_data = []
    for game_num in range(num_games):
        #print(f'Starting game {game_num + 1}/{num_games}')
        pos = Initial_Position
        game_history = []
        done = False

        move_count = 0  # To track the number of moves in the game

        while not done:
            #print(f'Game {game_num + 1}, Move {move_count + 1}')
            root = MCTSNode(pos)
            mcts = MCTS(net)
            #print('Starting MCTS search')
            pi = mcts.search(root)

            # Check if pi is None
            if pi is None:
                # No legal moves, game over
                done, winner, _ = get_game_status(pos)
                break

            # Store state and policy vector
            state = encode_board(pos)
            game_history.append((state, pi))

            # Choose action (sample from pi)
            action_index = np.random.choice(len(pi), p=pi)
            action = mcts.index_to_action(action_index)
            #print(f'Chosen action: {action}')

            # Apply action and update game state
            pos = apply_action(pos, action)

            # Check if game is over
            done, winner, _ = get_game_status(pos)
            move_count += 1

        #print(f'Finished game {game_num + 1}/{num_games} in {move_count} moves')

        # Assign outcomes to game history
        outcome = 0
        if winner == 1:
            outcome = 1
        elif winner == 2:
            outcome = -1
        for i in range(len(game_history)):
            state, pi = game_history[i]
            game_history[i] = (state, pi, outcome)
            outcome = -outcome  # Switch perspective

        training_data.extend(game_history)

    return training_data

# Training function with SGD and annealed learning rate
def train_network(net, training_data, batch_size=BATCH_SIZE, epochs=10):
    optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Anneal every 5 epochs
    net.train()
    policy_losses = []
    value_losses = []

    for epoch in range(epochs):
        np.random.shuffle(training_data)
        total_loss_p = 0
        total_loss_v = 0
        num_batches = 0

        for i in range(0, len(training_data), batch_size):
            batch = training_data[i:i+batch_size]
            states, pis, outcomes = zip(*batch)
            states = torch.tensor(states).float()
            pis = torch.tensor(pis).float()
            outcomes = torch.tensor(outcomes).float()

            optimizer.zero_grad()
            pred_pis, pred_vs = net(states)

            # Compute losses
            loss_p = -torch.sum(pis * torch.log(pred_pis + 1e-8)) / len(states)
            loss_v = torch.mean((pred_vs.view(-1) - outcomes) ** 2)
            loss = loss_p + loss_v

            loss.backward()
            optimizer.step()

            total_loss_p += loss_p.item()
            total_loss_v += loss_v.item()
            num_batches += 1

        avg_loss_p = total_loss_p / num_batches
        avg_loss_v = total_loss_v / num_batches
        policy_losses.append(avg_loss_p)
        value_losses.append(avg_loss_v)

        # Update learning rate
        scheduler.step()
        #print(f'Epoch {epoch+1}/{epochs}, Policy Loss: {avg_loss_p:.4f}, Value Loss: {avg_loss_v:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}')

    # Plot the losses
    #plt.figure(figsize=(12, 5))
    #plt.subplot(1, 2, 1)
    #plt.plot(policy_losses, label='Policy Loss')
    #plt.xlabel('Epoch')
    #plt.ylabel('Loss')
    #plt.title('Policy Loss over Epochs')
    #plt.legend()

    #plt.subplot(1, 2, 2)
    #plt.plot(value_losses, label='Value Loss')
    #plt.xlabel('Epoch')
    #plt.ylabel('Loss')
    #plt.title('Value Loss over Epochs')
    #plt.legend()

    #plt.show()

# Training function with SGD and annealed learning rate
def evaluate_network(Initial_Position, net, num_games=10):
    """
    Evaluate the performance of the neural network by having it play against a random player.
    Returns the win rate of the network.
    """
    net.eval()  # Ensure the network is in evaluation mode
    net_wins = 0
    draws = 0
    for game_num in range(num_games):
        print(f'\nStarting evaluation game {game_num + 1}/{num_games}')
        pos = Initial_Position
        done = False
        current_player = pos[16]
        move_count = 0
        position_counts = {}  # For repetition detection
        while not done:
            if current_player == 1:
                # Neural network's turn
                root = MCTSNode(pos)
                mcts = MCTS(net)
                pi = mcts.search(root)
                if pi is not None:
                    action_index = np.argmax(pi)
                    action = mcts.index_to_action(action_index)
                    pos = apply_action(pos, action)
                else:
                    # No legal moves, game over
                    done, winner, reason = get_game_status(pos)
                    #print(f'Game over: {reason}')
                    break
            else:
                # Random player's turn
                legal_actions = legal_moves(pos)
                if legal_actions:
                    action = random.choice(list(legal_actions.keys()))
                    pos = apply_action(pos, action)
                else:
                    # No legal moves, game over
                    done, winner, reason = get_game_status(pos)
                    #print(f'Game over: {reason}')
                    break
            # Update game state
            done, winner, reason = get_game_status(pos)
            current_player = pos[16]
            move_count += 1

            # Repetition detection
            pos_tuple = tuple(pos)
            position_counts[pos_tuple] = position_counts.get(pos_tuple, 0) + 1
            if position_counts[pos_tuple] >= 3:
                #print('Threefold repetition detected, ending game as draw.')
                done = True
                winner = 0  # Declare a draw
                break

            if move_count >= 100:
                #print('Reached move limit, ending game as draw.')
                done = True
                winner = 0  # Declare a draw
                break

        if winner == 1:
            net_wins += 1
        elif winner == 0:
            draws += 1
        #print(f'Finished evaluation game {game_num + 1}/{num_games}')

    win_rate = net_wins / num_games
    draw_rate = draws / num_games
    #print(f'Network win rate over {num_games} games: {win_rate:.2f}, Draw rate: {draw_rate:.2f}')
    return win_rate


### Main Functions ###

In [ ]:
# Function to display the board position
Decoder = [" ", chr(9817), chr(9814), chr(9816), chr(9815), chr(9813), chr(9812),chr(9823),chr(9820),chr(9822), chr(9821),chr(9819),chr(9818)]

def display_position(pos):
    x=math.sqrt(len(pos)-1)

    if x!=x//1 or x==0:

        print("Invalid Position")

    else:
        print(pos)
        i=0
        while i<x:
            s=""
            j=0
            while j<x:
                s = s + str(Decoder[pos[int(j+i*x)]]) + " "
                j = j + 1
            print(s)
            i = i + 1
        if(pos[int(x**2)]==1):
            print("White to Move")
        elif(pos[int(x**2)]==2):
            print("Black to Move")

def sample_training(pos):

    # Initialize the neural network
    net = AlphaZeroNet(ACTION_SIZE)

    # Try to load the previous model
    model_filename = 'best_alpha_zero_net.pth'
    best_win_rate = 0.0  # Initialize best win rate

    if os.path.exists(model_filename):
        net.load_state_dict(torch.load(model_filename, weights_only=True))
        print('Loaded previous model from file.')
        # Evaluate the loaded model
        best_win_rate = evaluate_network(pos, net, num_games=10)
    else:
        print('No previous model found, starting with a new model.')

    num_iterations = 1  # Adjust as needed
    for i in range(num_iterations):
        print(f'--- Iteration {i+1}/{num_iterations} ---')

        # Self-play to generate training data
        num_self_play_games = 5  # Adjust as needed
        training_data = self_play(pos, num_self_play_games, net)

        # Train the neural network
        train_network(net, training_data, batch_size=32, epochs=5)

        # Evaluate the neural network
        win_rate = evaluate_network(pos, net, num_games=5)

        # Compare with best win rate
        if win_rate > best_win_rate:
            print(f'New model is better (win rate: {win_rate:.2f} > {best_win_rate:.2f}), saving the model.')
            best_win_rate = win_rate
            # Save the new best model
            torch.save(net.state_dict(), model_filename)
        else:
            print(f'New model is not better (win rate: {win_rate:.2f} <= {best_win_rate:.2f}), keeping the previous model.')
            # Reload the previous best model
            net.load_state_dict(torch.load(model_filename, weights_only=True))

def best_move_calculator(pos, model_path):
    net = AlphaZeroNet(ACTION_SIZE)
    # Load the trained model
    net.load_state_dict(torch.load(model_path, weights_only=True))
    net.eval()  # Set the network to evaluation mode

    # Initialize MCTS for the current move
    root = MCTSNode(pos)
    mcts = MCTS(net)
    pi = mcts.search(root)
    #mcts.print_child_node_visits(root)

    # If no legal moves are available, the game is over
    if pi is None:
        done, winner, reason = get_game_status(pos)
        print(f'Game over: {reason}')

    # Select the action with the highest probability and apply it
    action_index = np.argmax(pi)
    action = mcts.index_to_action(action_index)

    print(f'Player {pos[16]} chooses action: {action}')
    pos = apply_action(pos, action)  # Update the position with the chosen action

    # Check the game status after applying the move
    done, winner, reason = get_game_status(pos)
    if done:
        print(f'Game over: {reason}')

    return pos

def simulate_game_from_position(pos, model_path):

    # Initialize the neural network
    net = AlphaZeroNet(ACTION_SIZE)
    # Load the trained model
    net.load_state_dict(torch.load(model_path, weights_only=True))

    net.eval()  # Ensure the network is in evaluation mode
    move_count = 0
    position_counts = {}  # For repetition detection
    first_move = None  # To save the first move of the game

    while True:
        print(f'\nMove {move_count + 1}')
        display_position(pos)

        # Initialize MCTS for the current move
        root = MCTSNode(pos)
        mcts = MCTS(net)
        pi = mcts.search(root)
        mcts.print_child_node_visits(root)

        # If no legal moves are available, the game is over
        if pi is None:
            done, winner, reason = get_game_status(pos)
            print(f'Game over: {reason}')
            break

        # Select the action with the highest probability and apply it
        action_index = np.argmax(pi)
        action = mcts.index_to_action(action_index)

        # Save the first move
        if move_count == 0:
            first_move = action
            print(f'Saving the first move: {first_move}')

        print(f'Player {pos[16]} chooses action: {action}')
        pos = apply_action(pos, action)  # Update the position with the chosen action

        # Check the game status after applying the move
        done, winner, reason = get_game_status(pos)
        if done:
            print(f'Game over: {reason}')
            break

        # Repetition detection
        pos_tuple = tuple(pos)
        position_counts[pos_tuple] = position_counts.get(pos_tuple, 0) + 1
        if position_counts[pos_tuple] >= 3:
            print('Threefold repetition detected, ending game as draw.')
            winner = 0  # Declare a draw
            break

        # Move limit condition
        if move_count >= 100:
            print('Reached move limit, ending game as draw.')
            winner = 0  # Declare a draw
            break

        # Increment the move counter
        move_count += 1

    # Display the final game result
    if winner == 1:
        print('White wins!')
        display_position(pos)
    elif winner == 2:
        print('Black wins!')
        display_position(pos)
    else:
        print('Game is a draw.')
        display_position(pos)
    return first_move, winner

### Sample Usage ###

In [ ]:
sample_pos = [12, 0, 0, 0, 7, 0, 1, 3, 1, 0, 0, 0, 4, 6, 0, 0, 2]
train_data_path = 'best_alpha_zero_net.pth'
sample_training(sample_pos)
best_move_calculator(sample_pos, train_data_path)
simulate_game_from_position(sample_pos, train_data_path)